In [1]:
!pip install scikit-learn

In [3]:
!pip install pandas numpy scikit-learn statsmodels

In [4]:
import pandas as pd
import numpy as np
from scipy.sparse import issparse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.api as sm

In [11]:
df = pd.read_csv("tabel_final.csv")
print(df.columns)

Index(['idSchool', 'Anul Scolar', 'Numele Scolii', 'Tipul Scolii',
       'Programul Scolii', 'Forma de proprietate', 'Judetul', 'Localitatea',
       'email', 'numele_scolii', 'tipul_clasei', 'numar_calculatoare',
       'numar_locuri', 'anul_achizitiei', 'Media'],
      dtype='object')


### Identificarea variabilelor relevante

##### Prima etapă în feature engineering este de a identifica variabilele care sunt relevante pentru problema pe care o analizezi. Variabile relevante identificate:
##### Media: Media elevului, care ar putea fi variabila țintă pentru analizele de performanță ale elevilor.
###### numar_locuri: Numărul locurilor, care ar putea indica capacitatea sălilor de clasă sau mijloacele de transport.
###### Numar_calculatoare: Numărul de calculatoare disponibile, care poate fi un indicator al resurselor educaționale ale școlii.
###### Anul_achizitiei: Anul achiziției, care ar putea indica vechimea echipamentelor sau a infrastructurii.
###### suprafata: Suprafața sălii, care ar putea fi un factor în confortul elevilor și eficiența învățării.


In [32]:
# Înlocuirea '-' cu NaN pentru a putea face conversii numerice
df.replace('-', np.nan, inplace=True)

In [13]:
# Conversia coloanelor în tipuri numerice și tratamentul valorilor NaN
numeric_cols = ['numar_calculatoare', 'numar_locuri', 'anul_achizitiei', 'Media']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [14]:
# Curățarea datelor după conversie
df.dropna(subset=numeric_cols, inplace=True)

### Feature Engineering

##### Creează noi caracteristici (features), inclusiv 'rata_calculatoare' (raportul dintre numărul de calculatoare și numărul de locuri) și 'vechime_calculatoare' (diferența dintre anul curent, presupus 2019, și anul achiziției calculatoarelor).

In [15]:
# Feature Engineering
df['rata_calculatoare'] = df['numar_calculatoare'] / df['numar_locuri']
df['vechime_calculatoare'] = 2019 - df['anul_achizitiei']
df['indice_modernitate_tehnologie'] = (2019 - df['anul_achizitiei']) * df['numar_calculatoare']
df['raport_facilitati_scoala'] = df['numar_calculatoare'] / df['numar_locuri']

In [17]:
# Identificarea caracteristicilor numerice și categorice
features_numerice = ['numar_calculatoare', 'numar_locuri', 'anul_achizitiei', 'rata_calculatoare', 'vechime_calculatoare',
                     'indice_modernitate_tehnologie', 'raport_facilitati_scoala']
features_categorice = ['Tipul Scolii', 'Judetul']

##### Indicele de modernitate a tehnologiei - Acest indice ar putea fi creat pentru a evalua modernitatea echipamentelor tehnologice (calculatoare) din școli. Acest indice ar putea fi calculat prin combinarea 'anul_achizitiei' cu 'numar_calculatoare', de exemplu, prin ponderarea numărului de calculatoare cu vechimea acestora. Un calcul simplu ar putea fi 1 / (Anul Curent - anul_achizitiei) * numar_calculatoare. Acest indice va da mai multă greutate școlilor cu echipamente mai noi. Presupunând că echipamentele tehnologice mai noi sunt potențial mai eficiente sau mai adecvate pentru nevoile educaționale curente, acest indice ar putea oferi o perspectivă mai bună asupra influenței tehnologiei asupra performanței școlare.

##### Raportul dintre Facilitățile Școlare și Mărimea Școlii - Acest raport ar putea fi definit ca numărul de calculatoare per elev sau per clasă (dacă informația despre numărul de elevi sau clase este disponibilă). Dacă aceste date nu sunt disponibile, se poate folosi 'numar_locuri' ca proxy pentru mărimea școlii. Formula ar putea fi numar_calculatoare / numar_locuri. Acest raport ar putea ajuta la înțelegerea modului în care resursele sunt distribuite în raport cu dimensiunea școlii și ar putea avea o influență asupra performanței educaționale.


### Procesarea datelor

##### Aplicarea standardizarii (scalarea) caracteristicilor numerice și transformarea One-Hot Encoding pentru variabilele categorice.

##### Am folosit ColumnTransformer pentru a combina aceste transformări într-un singur proces

In [18]:
# Crearea unui ColumnTransformer pentru preprocesare
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_numerice),
        ('cat', OneHotEncoder(), features_categorice)
    ]
)


In [19]:
# Separarea setului de date în caracteristici și țintă
X = df.drop('Media', axis=1)
Y = df['Media']

In [20]:
# Separarea datelor în seturi de antrenament și test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [37]:
# Aplicarea preprocesarea pe setul de antrenament și transformă datele într-un format compatibil pentru modelare.
X_train_preprocessed = preprocessor.fit_transform(X_train)

In [24]:
# Convertim matricea sparse într-o matrice dense înainte de a adăuga constanta
X_train_preprocessed = X_train_preprocessed.toarray() if issparse(X_train_preprocessed) else X_train_preprocessed
X_train_preprocessed = sm.add_constant(X_train_preprocessed)

##### Model de regresie liniară folosind statsmodels.OLS portivit pe setul de antrenament.

In [36]:
model = sm.OLS(Y_train, X_train_preprocessed).fit()

In [27]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Media   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     15.59
Date:                Fri, 24 Nov 2023   Prob (F-statistic):          1.35e-103
Time:                        19:20:17   Log-Likelihood:                -3504.4
No. Observations:                2347   AIC:                             7103.
Df Residuals:                    2300   BIC:                             7374.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0221      0.055     -0.403      0.6

### Rezultate regresie

##### R-squared (R²): 0.238. Acesta indică faptul că modelul explică aproximativ 23.8% din variația variabilei dependente (Media). Nu este un scor foarte mare, dar este un indicator că modelul are o anumită capacitate predictivă.
##### Adjusted R-squared: 0.2212. Acesta este R-squared ajustat pentru numărul de predictori în model și indică o potrivire decentă a modelului.
##### F-statistic: 15.59, cu un Prob (F-statistic) foarte mic (1.35e-103), indică faptul că cel puțin unele dintre variabilele independente au o relație semnificativă cu variabila dependentă.
##### Coeficienți: Fiecare x reprezintă un coeficient pentru o caracteristică (sau pentru o categorie dintr-o caracteristică categorică, după codificarea One-Hot). Coeficienții pozitivi sugerează o relație directă cu variabila dependentă, în timp ce coeficienții negativi indică o relație inversă.
##### P>|t| (p-value): p-value-uri asociate coeficienților indică dacă relațiile respective sunt statistic semnificative. De exemplu, un coeficient cu un p-value mai mic decât 0.05 este considerat a avea o influență semnificativă.
##### Omnibus, Skew, Kurtosis, Durbin-Watson, Jarque-Bera: Aceste teste și statisticile sunt legate de distribuția reziduurilor (erorilor) modelului. Ideal, vrem ca reziduurile să aibă o distribuție aproximativ normală.

In [28]:
# Preprocesarea și evaluarea modelului pe setul de test
X_test_preprocessed = preprocessor.transform(X_test)

In [29]:
# Convertim matricea sparse într-o matrice dense înainte de a adăuga constanta
X_test_preprocessed = X_test_preprocessed.toarray() if issparse(X_test_preprocessed) else X_test_preprocessed
X_test_preprocessed = sm.add_constant(X_test_preprocessed)

In [30]:
# Predicțiile pe setul de test
Y_pred = model.predict(X_test_preprocessed)


In [34]:
# Calculul metricilor de performanță
r2 = r2_score(Y_test, Y_pred)
print(f'R-squared (R²): {r2}')
mae = mean_absolute_error(Y_test, Y_pred)
print(f'Mean Absolute Error (MAE): {mae}')
mse = mean_squared_error(Y_test, Y_pred)
print(f'Mean Squared Error (MSE): {mse}')
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')

R-squared (R²): 0.22079922483903947
Mean Absolute Error (MAE): 0.8487018613427033
Mean Squared Error (MSE): 1.1509010130791695
Root Mean Squared Error (RMSE): 1.072800546736983


#### Metrici de Evaluare
##### R-squared (R²): 0.22. Aproape similar cu cel din setul de antrenament, indicând consistență între seturile de antrenament și test.
##### Mean Absolute Error (MAE): 0.847. Aceasta indică eroarea medie absolută a predicțiilor modelului.
##### Mean Squared Error (MSE): 1.150. Aceasta este media erorilor pătratice, care pune mai multă greutate pe erorile mari.
##### Root Mean Squared Error (RMSE): 1.0728. Este rădăcina pătrată a MSE și oferă o interpretare a erorii în aceleași unități ca variabila dependentă.
